In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkml").getOrCreate()

In [2]:
spark

In [51]:
#Data1

In [10]:
path0 = 's3://502pubg/Aggeragate/agg_match_stats_0.csv'
path1 = 's3://502pubg/Aggeragate/agg_match_stats_1.csv'
path2 = 's3://502pubg/Aggeragate/agg_match_stats_2.csv'
path3 = 's3://502pubg/Aggeragate/agg_match_stats_3.csv'
path4 = 's3://502pubg/Aggeragate/agg_match_stats_4.csv'

In [11]:
agg0 = spark.read.csv(path0, header=True, inferSchema=True)
agg1 = spark.read.csv(path1, header=True, inferSchema=True)
agg2 = spark.read.csv(path2, header=True, inferSchema=True)
agg3 = spark.read.csv(path3, header=True, inferSchema=True)
agg4 = spark.read.csv(path4, header=True, inferSchema=True)

In [16]:
agg = agg0.union(agg1)
agg = agg.union(agg2)
agg = agg.union(agg3)
agg = agg.union(agg4)

In [17]:
agg.count()

67369231

In [18]:
agg.printSchema()

root
 |-- date: string (nullable = true)
 |-- game_size: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_placement: integer (nullable = true)



In [19]:
agg.show()

+--------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+--------------+-------------------+-------+--------------+
|                date|game_size|            match_id|match_mode|party_size|player_assists|player_dbno|  player_dist_ride|  player_dist_walk|player_dmg|player_kills|   player_name|player_survive_time|team_id|team_placement|
+--------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+--------------+-------------------+-------+--------------+
|2017-11-26T20:59:...|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|          2870.724|        1784.84778|       117|           1|      SnuffIes|            1106.32|      4|            18|
|2017-11-26T20:59:...|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|2938.4

In [22]:
agg=agg.drop('date')
agg =agg.drop('match_id')
agg = agg.drop('player_name')
agg = agg.drop('team_id')

In [23]:
agg.printSchema()

root
 |-- game_size: integer (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_placement: integer (nullable = true)



In [30]:
from pyspark.sql.types import IntegerType

In [31]:
agg_0 = agg.withColumn('player_dist_ride',agg['player_dist_ride'].cast(IntegerType()))
agg_0 = agg_0.withColumn('player_dist_walk',agg['player_dist_walk'].cast(IntegerType()))
agg_0 = agg_0.withColumn('player_survive_time',agg['player_survive_time'].cast(IntegerType()))

In [32]:
agg_0.printSchema()

root
 |-- game_size: integer (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: integer (nullable = true)
 |-- player_dist_walk: integer (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_survive_time: integer (nullable = true)
 |-- team_placement: integer (nullable = true)



In [34]:
agg_0.na.drop().count()

67369231

In [40]:
agg_0.write.parquet('s3://502pubg/clean/aggeragate0.parquet')

In [24]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler

In [25]:
indexer1 = StringIndexer(inputCol='match_mode', outputCol="matchmode_index")

In [45]:
agg_1 = indexer1.fit(agg_0).transform(agg_0)

In [46]:
indexer2 = StringIndexer(inputCol='team_placement', outputCol="label")
agg_1 = indexer2.fit(agg_1).transform(agg_1)

In [48]:
agg_1.printSchema()

root
 |-- game_size: integer (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: integer (nullable = true)
 |-- player_dist_walk: integer (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_survive_time: integer (nullable = true)
 |-- team_placement: integer (nullable = true)
 |-- matchmode_index: double (nullable = false)
 |-- label: double (nullable = false)



In [49]:
agg_1.show()

+---------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-------------------+--------------+---------------+-----+
|game_size|match_mode|party_size|player_assists|player_dbno|player_dist_ride|player_dist_walk|player_dmg|player_kills|player_survive_time|team_placement|matchmode_index|label|
+---------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-------------------+--------------+---------------+-----+
|       37|       tpp|         2|             0|          1|            2870|            1784|       117|           1|               1106|            18|            0.0| 15.0|
|       37|       tpp|         2|             0|          1|            2938|            1756|       127|           1|               1106|            18|            0.0| 15.0|
|       37|       tpp|         2|             0|          0|               0|             224|        67|           0|  

In [50]:
agg_1.write.parquet('s3://502pubg/clean/aggeragate1.parquet')

Data2

In [52]:
path0 = 's3://502pubg/deaths/kill_match_stats_final_0.csv'
path1 = 's3://502pubg/deaths/kill_match_stats_final_1.csv'
path2 = 's3://502pubg/deaths/kill_match_stats_final_2.csv'
path3 = 's3://502pubg/deaths/kill_match_stats_final_3.csv'
path4 = 's3://502pubg/deaths/kill_match_stats_final_4.csv'

In [54]:
kill0 = spark.read.csv(path0, header=True, inferSchema=True)
kill1 = spark.read.csv(path1, header=True, inferSchema=True)
kill2 = spark.read.csv(path2, header=True, inferSchema=True)
kill3 = spark.read.csv(path3, header=True, inferSchema=True)
kill4 = spark.read.csv(path4, header=True, inferSchema=True)

In [55]:
kill0.count()

13426348

In [56]:
kill = kill0.union(kill1)
kill = kill.union(kill2)
kill = kill.union(kill3)
kill = kill.union(kill4)

In [57]:
kill.printSchema()

root
 |-- killed_by: string (nullable = true)
 |-- killer_name: string (nullable = true)
 |-- killer_placement: double (nullable = true)
 |-- killer_position_x: double (nullable = true)
 |-- killer_position_y: double (nullable = true)
 |-- map: string (nullable = true)
 |-- match_id: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- victim_name: string (nullable = true)
 |-- victim_placement: double (nullable = true)
 |-- victim_position_x: double (nullable = true)
 |-- victim_position_y: double (nullable = true)



In [58]:
kill.show()

+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|   killed_by|     killer_name|killer_placement|killer_position_x|killer_position_y|    map|            match_id|time|    victim_name|victim_placement|victim_position_x|victim_position_y|
+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|     Grenade| KrazyPortuguese|             5.0|         657725.1|         146275.2|MIRAMAR|2U4GBNA0YmnLSqvEy...| 823|KrazyPortuguese|             5.0|         657725.1|         146275.2|
|      SCAR-L|nide2Bxiaojiejie|            31.0|         93091.37|         722236.4|MIRAMAR|2U4GBNA0YmnLSqvEy...| 194|    X3evolution|            33.0|         92238.68|         723375.1|
|        S686|        Ascholes|            43.0|         366

In [59]:
kill.count()

65370475

In [60]:
kill = kill.drop('killer_name')
kill = kill.drop('match_id')
kill = kill.drop('victim_name')

In [61]:
kill.printSchema()

root
 |-- killed_by: string (nullable = true)
 |-- killer_placement: double (nullable = true)
 |-- killer_position_x: double (nullable = true)
 |-- killer_position_y: double (nullable = true)
 |-- map: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- victim_placement: double (nullable = true)
 |-- victim_position_x: double (nullable = true)
 |-- victim_position_y: double (nullable = true)



In [62]:
kill_0 = kill.withColumn('killer_position_x',kill['killer_position_x'].cast(IntegerType()))
kill_0 = kill_0.withColumn('killer_position_y',kill['killer_position_y'].cast(IntegerType()))
kill_0 = kill_0.withColumn('victim_position_x',kill['victim_position_x'].cast(IntegerType()))
kill_0 = kill_0.withColumn('victim_position_y',kill['victim_position_y'].cast(IntegerType()))

In [63]:
kill_0.printSchema()

root
 |-- killed_by: string (nullable = true)
 |-- killer_placement: double (nullable = true)
 |-- killer_position_x: integer (nullable = true)
 |-- killer_position_y: integer (nullable = true)
 |-- map: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- victim_placement: double (nullable = true)
 |-- victim_position_x: integer (nullable = true)
 |-- victim_position_y: integer (nullable = true)



In [65]:
kill_0.write.parquet('s3://502pubg/clean/kill0.parquet')